In [1]:
# f = open('./input.txt','r').read().strip()
# pipeMap = [res for res in f.split('\n')]
# placeholder = 99
# pathMap = [[placeholder] * len(pipeMap[0]) for _ in range(len(pipeMap))]
# d = {'-':[(0,-1),(0,1)],'|':[(1,0),(-1,0)],'7':[(1,0),(0,-1)],'F':[(1,0),(0,1)],'J':[(-1,0),(0,-1)],'L':[(-1,0),(0,1)],'.':[]}

# sy,sx = 0,0
# for i,l in enumerate(pipeMap):
#     for j,e in enumerate(l):
#         if e == 'S':
#             sy,sx = i,j

# pathMap[sy][sx] = 0

# matches = []
# directions = [(-1,0),(0,1),(1,0),(0,-1)]
# for k in d.keys():
#     counter = 0
#     for dy,dx in directions:
#         if (dy,dx) in d[k] and (-dy,-dx) in d[pipeMap[sy+dy][sx+dx]]:
#             counter += 1
#     matches.append((k,counter))

# openList = [(sy+a,sx+b) for a,b in d[sorted(matches,key=lambda x: x[1])[-1][0]]]
# for y,x in openList:
#     pathMap[y][x] = 1
# closedList = [(sy,sx)]

# while len(openList) > 0:
#     y,x = openList.pop()
#     cost = pathMap[y][x]+1
#     for ry,rx in d[pipeMap[y][x]]:
#         if pathMap[y+ry][x+rx] <= cost:
#             continue
#         else:
#             openList.append((y+ry,x+rx))
#             pathMap[y+ry][x+rx] = cost
#     # closedList.append((y,x))
# res = 0
# for l in pathMap:
#     for e in l:
#         res = e if e > res and e != placeholder else res
# print(res)
# for l in pathMap:
#     print(f'{l}\n')

# Part 2
import numpy as np
np.set_printoptions(suppress=True, linewidth=100000)

def replaceS(y,x,d,map):
    matches = []
    directions = [(-1,0),(0,1),(1,0),(0,-1)]
    for k in d.keys():
        counter = 0
        for dy,dx in directions:
            if (dy,dx) in d[k] and (-dy,-dx) in d[map[y+dy][x+dx]]:
                counter += 1
        matches.append((k,counter))
    return sorted(matches,key=lambda x: x[1])[-1][0]

f = open('./input.txt','r').read().strip()
pipeMap = [res for res in f.split('\n')]
placeholder = 99
pathMap = [[placeholder] * len(pipeMap[0]) for _ in range(len(pipeMap))]
d = {'-':[(0,-1),(0,1)],'|':[(1,0),(-1,0)],'7':[(1,0),(0,-1)],'F':[(1,0),(0,1)],'J':[(-1,0),(0,-1)],'L':[(-1,0),(0,1)],'S':[],'*':[],'.':[]}

sy,sx = 0,0
for i,l in enumerate(pipeMap):
    for j,e in enumerate(l):
        if e == 'S':
            sy,sx = i,j

pathMap[sy][sx] = 0
openList = [(sy+a,sx+b) for a,b in d[replaceS(sy,sx,d,pipeMap)]]
for y,x in openList:
    pathMap[y][x] = 1
closedList = [(sy,sx)]

while len(openList) > 0:
    y,x = openList.pop()
    cost = pathMap[y][x]+1
    for ry,rx in d[pipeMap[y][x]]:
        if pathMap[y+ry][x+rx] <= cost:
            continue
        else:
            openList.append((y+ry,x+rx))
            pathMap[y+ry][x+rx] = cost
    # closedList.append((y,x))

# SDF = np.zeros((len(pathMap),len(pathMap[0])),dtype=str)
SDF = np.full((len(pathMap),len(pathMap[0])),'*',dtype=str)

for i,l in enumerate(pathMap):
    for j,e in enumerate(l):
        if e != 99:
            SDF[i,j] = pipeMap[i][j]

for i,l in enumerate(SDF):
    for j,e in enumerate(l):
        if e == 'S':
            SDF[i,j] = replaceS(i,j,d,SDF)

for i,l in enumerate(SDF):
    for j,e in enumerate(l):
        if SDF[i][j-1] in ['J','7','|'] and SDF[i][j] in ['F','|','L']:
            a = ['S' for _ in range(len(SDF))]
            a[0] = '*'
            a[-1] = '*'
            SDF = np.insert(SDF,j,a,axis=1)

for i,l in enumerate(SDF):
    for j,e in enumerate(l):
        if e == 'S':
            SDF[i,j] = replaceS(i,j,d,SDF)
SDF = np.pad(SDF,1,constant_values='/')

stars = []
for i,l in enumerate(SDF):
    for j,e in enumerate(l):
        if e == '*':
            stars.append((i,j))

for sy,sx in stars:
    openList = [(sy,sx)]
    closedList = []
    looping = True
    while looping:
        if len(openList) == 0:
            SDF[sy,sx] = 'I'
            break
        y,x = openList.pop()
        closedList.append((y,x))
        for ry,rx in [(-1,0),(0,1),(1,0),(0,-1)]:
            if SDF[y+ry,x+rx] == '/':
                SDF[sy,sx] = 'O'
                looping = False
                break
            elif SDF[y+ry,x+rx] in ['*','.','O','I'] and (y+ry,x+rx) not in closedList:
                openList.append((y+ry,x+rx))
            else:
                continue
print(SDF)

res = 0
for i,l in enumerate(SDF):
    for j,e in enumerate(l):
        if e == 'I':
            res += 1
print(res)

print(np.repeat([1],1000))

[['/' '/' '/' '/' '/' '/' '/' '/' '/' '/' '/' '/' '/' '/' '/' '/' '/' '/'
  '/' '/' '/' '/' '/' '/' '/' '/' '/' '/' '/' '/' '/' '/']
 ['/' 'O' 'F' 'O' '-' '-' 'O' '-' '-' 'O' '7' 'O' 'F' 'O' '7' 'O' 'F' 'O'
  '7' 'O' 'F' 'O' '7' 'O' 'F' '-' '7' 'O' 'O' 'O' 'O' '/']
 ['/' 'O' '|' '.' 'F' '-' '-' '-' '7' '.' '|' '.' '|' '.' '|' '.' '|' '.'
  '|' '.' '|' '.' '|' '.' '|' 'F' 'J' 'O' 'O' 'O' 'O' '/']
 ['/' 'O' '|' '.' '|' 'O' '.' 'F' 'J' '.' '|' '.' '|' '.' '|' '.' '|' '.'
  '|' '.' '|' '.' '|' '.' '|' 'L' '7' 'O' 'O' 'O' 'O' '/']
 ['/' 'F' 'J' '.' 'L' '7' '.' 'L' '7' '.' 'L' '-' 'J' '.' 'L' '-' 'J' '.'
  '|' '.' '|' '.' 'L' '-' 'J' 'I' 'L' '-' '7' 'O' 'O' '/']
 ['/' 'L' '-' '-' '-' 'J' '.' 'O' 'L' '-' '7' '.' 'O' '.' 'O' '.' 'O' '.'
  'L' '-' 'J' '.' 'F' '-' '7' 'F' '-' '7' 'L' '7' 'O' '/']
 ['/' 'O' 'O' '.' 'O' 'O' '.' 'F' '-' '-' 'J' '.' 'O' '.' 'O' '.' 'F' '-'
  '7' '.' 'F' '-' 'J' '.' '|' 'L' '7' 'L' '7' 'L' '7' '/']
 ['/' 'O' 'O' '.' 'O' 'O' '.' 'L' '7' '.' 'O' '.' 'F' '-' '7' '.' '|'